In [184]:
from pytil.utility import *
from pytil.object import Namespace as O
import re
from string import Template


class BlocksLibrary(dict):
    def __init__(self):
        pass
    
    def __setitem__(self, key, val):
        raise TypeError("cannot assign to BlacksLibrary!")
    
    def __missing__(self, block_name):
        numspec = {}
        with open("block " + block_name + " .txt", 'r') as file:
            for line in file:
                items = line.strip().split()
                numspec[int(items[0])] = items[1:]
        dict.__setitem__(self, block_name, numspec)
        return numspec
    
blib = BlocksLibrary()
    
    
def line_head(line): # return heading information from line
    pref = line[:re.search("\w", line).span()[0]]
    nlvl = re.search("[^\t]", line).span()[0]
    return pref, nlvl
    
    
def build_blocks():
    blocks = []
    block = None
    with open("info .txt", 'r') as file:
        lineiter = iter(file)
        while True:
            try:
                line = next(lineiter).rstrip()
            except StopIteration:
                break
            if line[0] == "{": # new block
                name = line[1:-1]
                if block is not None:
                    blocks.append(block)
                block = O(type=name, raw=[])
                block.title = next(lineiter).rstrip()
                node = block
                while True:
                    try:
                        line = next(lineiter).rstrip()
                    except StopIteration:
                        line = None
                    if line == '[]' or line == '' or line is None:
                        nprimary = min((i for i, line in enumerate(node.raw) if line_head(line)[1] == 2),
                                       default=len(node.raw))
                        primary, secondary = node.raw[:nprimary], node.raw[nprimary:]
                        for i, val in enumerate(primary):
                            node[blib[name][nprimary][i]] = val[1:]
                        node.points = [line[2:] for line in secondary]
                        # del node["raw"]
                        if line == '' or line is None:
                            break
                        node.next = O(type=name, raw=[])
                        node = node.next
                    else:
                        node.raw.append(line)
        if block is not None:
            blocks.append(block)
    return blocks
    
    
before_stuff = r"""
\documentclass[10pt,letterpaper]{article}
\usepackage[margin=0.75in]{geometry}
\usepackage{pifont}
\usepackage{hyperref}
\usepackage{titlesec} % Used to customize the \section command
\titleformat{\section}{\Large\scshape\raggedright}{}{0em}{}[\titlerule] %
\usepackage{enumitem}
\setitemize{itemsep=3pt,topsep=5pt,parsep=0pt,partopsep=0pt}

\newcommand*\textmb[1]{\fontseries{b}\selectfont#1\fontseries{m}\selectfont}
\newcommand*\textul[1]{\underline{\smash{#1}}}

\newcommand*\company[1]{\textbf{#1}}
\newcommand*\position[1]{\textmb{#1}}
\newcommand*\location\company
\newcommand*\timespan\position

\begin{document}
\pagestyle{empty}
"""

after_stuff = "\n" + r"\end{document}"


def texify_block(block):
    slist = []
    if block.type == "head":
        slist.append(r"\begin{center}")
        slist.append(Template(r"{\huge\textbf{$title}}").substitute(title=block.title) + "\n")
        slist.append(Template(r"$address\ \ \textbullet\ \ $phone\ \ \textbullet\ \ $email")
                     .substitute(address=block.address, phone=block.phone, email=block.email) + "\n")
        slist.append(Template(r"$linkedin\ \ \textbullet\ \ $github")
                     .substitute(linkedin=block.linkedin, github=block.github))
        slist.append(r"\end{center}")
    elif block.type == "section":
        def process_company(s):
            paren = s.find('()')
            if paren == -1:
                return r"\company{" + s + "}"
            else:
                return r"\company{" + s[: paren - 1] + "} (" + s[paren + 3 :] + ")"
        def process_position(s):
            paren = s.find('()')
            if paren == -1:
                return r"\position{" + s + "}"
            else:
                return r"\position{" + s[: paren - 1] + "} (" + s[paren + 3 :] + ")"                
        slist.append(Template(r"\section*{$title}").substitute(title=block.title))
        node = block
        while True:
            if hasattr(node, "company"):
                slist.append(Template(r"$company_processed\hfill\location{$location}\\")
                             .substitute(company_processed=process_company(node.company), location=node.location))
                slist.append(Template(r"$position_processed\hfill\timespan{$timespan}")
                             .substitute(position_processed=process_position(node.position), timespan=node.timespan))
            elif hasattr(node, "position"):
                slist.append(Template(r"$position_processed\hfill\timespan{$timespan}")
                             .substitute(position_processed=process_position(node.position), timespan=node.timespan))
            if node.points:
                slist.append(r"\begin{itemize}")
                for point in node.points:
                    slist.append(Template(r"\item $point").substitute(point=point))
                slist.append(r"\end{itemize}")
            if hasattr(node, "next"):
                node = node.next
            else:
                break
    return "\n".join(slist)


def make_tex(blocks):
    return before_stuff + "\n".join(texify_block(block) for block in blocks) + after_stuff


def compile_resume():
    with open("output.tex", 'w') as file:
        print(make_tex(build_blocks()), file=file)
    import subprocess
    subprocess.check_call(['pdflatex', 'output.tex'])

    
compile_resume()